# Feature engineering

In [ ]:
!pip install duckdb duckdb-engine jupysql odfpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 4.9 MB/s eta 0:00:00
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160673 sha256=d6d557e597a5da7b868f59923442f21bc3fed27c707fb22a1e63b74f525ebb81
  Stored in directory: /root/.cache/pip/wheels/36/5d/63/8243a7ee78fff0f944d638fd0e66d7278888f5e2285d7346b6
Successfully built odfpy


In [ ]:
import pandas as pd
import duckdb
from pprint import pprint

In [ ]:
BASE_PATH = '/content/drive/MyDrive/EyF/'
DATASET_PATH = BASE_PATH
MODELOS_PATH = BASE_PATH + 'modelos/'
DB_PATH = BASE_PATH + 'db/'
DATASET_FILE = 'trainset.csv'
NEW_FILENAME = 'fe_v2.csv'
DICCIONARY_FILENAME = 'DiccionarioDatos_2025.ods'
CRUDO_FILENAME = 'competencia_01_crudo.csv'

In [ ]:
dicc = pd.read_excel(BASE_PATH + DICCIONARY_FILENAME, index_col = 0, engine = 'odf').set_index("campo")

In [ ]:
lags = ['ctrx_quarter',
 'mcaja_ahorro',
 'mpasivos_margen',
 'mcuentas_saldo',
 'cpayroll_trx',
 'mprestamos_personales',
 'mtarjeta_visa_consumo',
 'mcuenta_corriente',
 'cprestamos_personales',
 'ctarjeta_visa_transacciones',
 'mrentabilidad_annual',
 'Visa_msaldopesos',
 'mpayroll',
 'Master_status',
 'mactivos_margen',
 'Visa_status',
 'ctarjeta_visa',
 'mcomisiones_mantenimiento',
 'ctarjeta_master',
 'Master_fechaalta',
 'tcallcenter',
 'cliente_edad',
 'mcomisiones',
 'Visa_msaldototal',
 'chomebanking_transacciones',
 'cproductos',
 'internet',
 'Master_Fvencimiento',
 'Visa_fechaalta',
 'ccallcenter_transacciones',
 'ccajas_consultas',
 'Master_mfinanciacion_limite',
 'mautoservicio',
 'mrentabilidad',
 'Visa_mpagospesos',
 'Visa_Fvencimiento',
 'ccomisiones_otras',
 'Visa_mfinanciacion_limite',
 'cliente_antiguedad',
 'cdescubierto_preacordado',
 'mtransferencias_recibidas',
 'Visa_mpagominimo',
 'ccomisiones_mantenimiento',
 'Visa_mconsumospesos',
 'mcomisiones_otras',
 'mcaja_ahorro_dolares',
 'Master_mlimitecompra',
 'cmobile_app_trx',
 'Master_mpagominimo',
 'ccajas_transacciones']

In [ ]:
print(len(lags))

50


# Exploración

In [ ]:
dicc.to_csv("dicc.csv")

In [ ]:
features = list(dicc.index)
print(features)

['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'csegur

# Conexión

In [ ]:
con = duckdb.connect()

# Clase ternaria

In [ ]:
query = f"""create or replace table competencia_01 as
select *
from read_csv_auto("{BASE_PATH + CRUDO_FILENAME}")"""

con.execute(query)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
pprint(query)

('create or replace table competencia_01 as\n'
 'select *\n'
 'from read_csv_auto("/content/drive/MyDrive/EyF/competencia_01_crudo.csv")')


In [ ]:
query = """CREATE TABLE competencia_01_ternaria AS (
WITH clientes AS (
    SELECT *
    FROM competencia_01
)
SELECT
    c1.*,
    CASE
        WHEN c2.numero_de_cliente IS NULL THEN 'BAJA+1'
        WHEN c2.numero_de_cliente IS NOT NULL AND c3.numero_de_cliente IS NULL THEN 'BAJA+2'
        ELSE 'CONTINUA'
    END AS clase_ternaria
FROM clientes c1
LEFT JOIN clientes c2
    ON c1.numero_de_cliente = c2.numero_de_cliente
    AND c2.foto_mes = c1.foto_mes + 1   -- cliente presente en el mes siguiente
LEFT JOIN clientes c3
    ON c1.numero_de_cliente = c3.numero_de_cliente
    AND c3.foto_mes = c1.foto_mes + 2   -- cliente presente en el segundo mes siguiente
--WHERE c1.foto_mes < 202106   -- últimos registros no se etiquetan
ORDER BY c1.numero_de_cliente, c1.foto_mes)"""

con.execute(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
query = """SELECT
foto_mes,
clase_ternaria,
count(*)
FROM competencia_01_ternaria
GROUP BY foto_mes, clase_ternaria
"""

prueba = con.execute(query).fetchdf()

In [ ]:
prueba.sort_values(by = ["foto_mes","clase_ternaria"]).set_index(["foto_mes","clase_ternaria"])

count_star()
foto_mes clase_ternaria              
202101   BAJA+1                   622
         BAJA+2                   825
         CONTINUA              160080
202102   BAJA+1                   831
         BAJA+2                  1032
         CONTINUA              160292
202103   BAJA+1                  1039
         BAJA+2                   951
         CONTINUA              161119
202104   BAJA+1                   955
         BAJA+2                  1130
         CONTINUA              161333
202105   BAJA+1                  1134
         BAJA+2                162783
202106   BAJA+1                164313

In [ ]:
query = """SELECT *
FROM competencia_01_ternaria
LIMIT 2
"""

prueba_ = con.execute(query).fetchdf()

In [ ]:
prueba_.columns.shape

(155,)

In [ ]:
vars_to_keep = list(prueba_.columns)

# Creación de la tabla

In [ ]:
str_important_features = ',\n'.join([feat for feat in vars_to_keep])

In [ ]:
# Crea la query

query = f"""create or replace table competencia_01 as
select *
from competencia_01_ternaria"""

con.execute(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# Lags

In [ ]:
no_lags = [
    'cliente_edad',
    'cliente_antiguedad',
    "numero_de_cliente",
    "foto_mes",
    "cliente_vip",
    "cprestamos_prendarios",
    "cprestamos_hipotecarios",
    "cseguro_vida",
    "cseguro_auto",
    "cseguro_vivienda",
    "cseguro_accidentes_personales",
    "Visa_fultimo_cierre",
    "Master_Finiciomora",
    "Visa_Finiciomora",
    "Master_delinquency",
    "Visa_delinquency",
    "active_quarter",
    "clase_ternaria",
    'Visa_fechaalta',
    'Master_fechaalta'
]

In [ ]:
lags = [v for v in lags if v not in no_lags]
print(lags)

['ctrx_quarter', 'mcaja_ahorro', 'mpasivos_margen', 'mcuentas_saldo', 'cpayroll_trx', 'mprestamos_personales', 'mtarjeta_visa_consumo', 'mcuenta_corriente', 'cprestamos_personales', 'ctarjeta_visa_transacciones', 'mrentabilidad_annual', 'Visa_msaldopesos', 'mpayroll', 'Master_status', 'mactivos_margen', 'Visa_status', 'ctarjeta_visa', 'mcomisiones_mantenimiento', 'ctarjeta_master', 'tcallcenter', 'mcomisiones', 'Visa_msaldototal', 'chomebanking_transacciones', 'cproductos', 'internet', 'Master_Fvencimiento', 'ccallcenter_transacciones', 'ccajas_consultas', 'Master_mfinanciacion_limite', 'mautoservicio', 'mrentabilidad', 'Visa_mpagospesos', 'Visa_Fvencimiento', 'ccomisiones_otras', 'Visa_mfinanciacion_limite', 'cdescubierto_preacordado', 'mtransferencias_recibidas', 'Visa_mpagominimo', 'ccomisiones_mantenimiento', 'Visa_mconsumospesos', 'mcomisiones_otras', 'mcaja_ahorro_dolares', 'Master_mlimitecompra', 'cmobile_app_trx', 'Master_mpagominimo', 'ccajas_transacciones']


In [ ]:
new_lags_variables = [f'lag_{i}_{lag_feature}' for lag_feature in lags for i in range(1,4)]
str_lags = ',\n'.join([f'lag({lag_feature}, {i}) over (partition by numero_de_cliente order by foto_mes) as lag_{i}_{lag_feature}' for lag_feature in lags for i in range(1,4)])

new_lags_variables

['lag_1_ctrx_quarter',
 'lag_2_ctrx_quarter',
 'lag_3_ctrx_quarter',
 'lag_1_mcaja_ahorro',
 'lag_2_mcaja_ahorro',
 'lag_3_mcaja_ahorro',
 'lag_1_mpasivos_margen',
 'lag_2_mpasivos_margen',
 'lag_3_mpasivos_margen',
 'lag_1_mcuentas_saldo',
 'lag_2_mcuentas_saldo',
 'lag_3_mcuentas_saldo',
 'lag_1_cpayroll_trx',
 'lag_2_cpayroll_trx',
 'lag_3_cpayroll_trx',
 'lag_1_mprestamos_personales',
 'lag_2_mprestamos_personales',
 'lag_3_mprestamos_personales',
 'lag_1_mtarjeta_visa_consumo',
 'lag_2_mtarjeta_visa_consumo',
 'lag_3_mtarjeta_visa_consumo',
 'lag_1_mcuenta_corriente',
 'lag_2_mcuenta_corriente',
 'lag_3_mcuenta_corriente',
 'lag_1_cprestamos_personales',
 'lag_2_cprestamos_personales',
 'lag_3_cprestamos_personales',
 'lag_1_ctarjeta_visa_transacciones',
 'lag_2_ctarjeta_visa_transacciones',
 'lag_3_ctarjeta_visa_transacciones',
 'lag_1_mrentabilidad_annual',
 'lag_2_mrentabilidad_annual',
 'lag_3_mrentabilidad_annual',
 'lag_1_Visa_msaldopesos',
 'lag_2_Visa_msaldopesos',
 'lag_3

In [ ]:
str_lags

'lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter,\nlag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter,\nlag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter,\nlag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro,\nlag(mcaja_ahorro, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcaja_ahorro,\nlag(mcaja_ahorro, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcaja_ahorro,\nlag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpasivos_margen,\nlag(mpasivos_margen, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpasivos_margen,\nlag(mpasivos_margen, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpasivos_margen,\nlag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by fot

In [ ]:
query = f"""create or replace table competencia_01 as
select *,
    {str_lags}
from competencia_01"""

In [ ]:
con.execute(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# delta_1

In [ ]:
delta_vars = [f"delta_1m_{lag_feature}" for lag_feature in lags]
delta_vars

['delta_1m_ctrx_quarter',
 'delta_1m_mcaja_ahorro',
 'delta_1m_mpasivos_margen',
 'delta_1m_mcuentas_saldo',
 'delta_1m_cpayroll_trx',
 'delta_1m_mprestamos_personales',
 'delta_1m_mtarjeta_visa_consumo',
 'delta_1m_mcuenta_corriente',
 'delta_1m_cprestamos_personales',
 'delta_1m_ctarjeta_visa_transacciones',
 'delta_1m_mrentabilidad_annual',
 'delta_1m_Visa_msaldopesos',
 'delta_1m_mpayroll',
 'delta_1m_Master_status',
 'delta_1m_mactivos_margen',
 'delta_1m_Visa_status',
 'delta_1m_ctarjeta_visa',
 'delta_1m_mcomisiones_mantenimiento',
 'delta_1m_ctarjeta_master',
 'delta_1m_tcallcenter',
 'delta_1m_mcomisiones',
 'delta_1m_Visa_msaldototal',
 'delta_1m_chomebanking_transacciones',
 'delta_1m_cproductos',
 'delta_1m_internet',
 'delta_1m_Master_Fvencimiento',
 'delta_1m_ccallcenter_transacciones',
 'delta_1m_ccajas_consultas',
 'delta_1m_Master_mfinanciacion_limite',
 'delta_1m_mautoservicio',
 'delta_1m_mrentabilidad',
 'delta_1m_Visa_mpagospesos',
 'delta_1m_Visa_Fvencimiento',
 '

In [ ]:
str_delta = ',\n'.join([
    f"{lag_feature} - lag_1_{lag_feature} as delta_1m_{lag_feature}"
    for lag_feature in lags
])
str_delta

'ctrx_quarter - lag_1_ctrx_quarter as delta_1m_ctrx_quarter,\nmcaja_ahorro - lag_1_mcaja_ahorro as delta_1m_mcaja_ahorro,\nmpasivos_margen - lag_1_mpasivos_margen as delta_1m_mpasivos_margen,\nmcuentas_saldo - lag_1_mcuentas_saldo as delta_1m_mcuentas_saldo,\ncpayroll_trx - lag_1_cpayroll_trx as delta_1m_cpayroll_trx,\nmprestamos_personales - lag_1_mprestamos_personales as delta_1m_mprestamos_personales,\nmtarjeta_visa_consumo - lag_1_mtarjeta_visa_consumo as delta_1m_mtarjeta_visa_consumo,\nmcuenta_corriente - lag_1_mcuenta_corriente as delta_1m_mcuenta_corriente,\ncprestamos_personales - lag_1_cprestamos_personales as delta_1m_cprestamos_personales,\nctarjeta_visa_transacciones - lag_1_ctarjeta_visa_transacciones as delta_1m_ctarjeta_visa_transacciones,\nmrentabilidad_annual - lag_1_mrentabilidad_annual as delta_1m_mrentabilidad_annual,\nVisa_msaldopesos - lag_1_Visa_msaldopesos as delta_1m_Visa_msaldopesos,\nmpayroll - lag_1_mpayroll as delta_1m_mpayroll,\nMaster_status - lag_1_Mast

In [ ]:
query = f"""create or replace table competencia_01 as
select *,
    {str_delta}
from competencia_01"""

In [ ]:
print(query)

create or replace table competencia_01 as
select *,
    ctrx_quarter - lag_1_ctrx_quarter as delta_1m_ctrx_quarter,
mcaja_ahorro - lag_1_mcaja_ahorro as delta_1m_mcaja_ahorro,
mpasivos_margen - lag_1_mpasivos_margen as delta_1m_mpasivos_margen,
mcuentas_saldo - lag_1_mcuentas_saldo as delta_1m_mcuentas_saldo,
cpayroll_trx - lag_1_cpayroll_trx as delta_1m_cpayroll_trx,
mprestamos_personales - lag_1_mprestamos_personales as delta_1m_mprestamos_personales,
mtarjeta_visa_consumo - lag_1_mtarjeta_visa_consumo as delta_1m_mtarjeta_visa_consumo,
mcuenta_corriente - lag_1_mcuenta_corriente as delta_1m_mcuenta_corriente,
cprestamos_personales - lag_1_cprestamos_personales as delta_1m_cprestamos_personales,
ctarjeta_visa_transacciones - lag_1_ctarjeta_visa_transacciones as delta_1m_ctarjeta_visa_transacciones,
mrentabilidad_annual - lag_1_mrentabilidad_annual as delta_1m_mrentabilidad_annual,
Visa_msaldopesos - lag_1_Visa_msaldopesos as delta_1m_Visa_msaldopesos,
mpayroll - lag_1_mpayroll as del

In [ ]:
con.execute(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
query = """select * from competencia_01 limit 5"""
df = con.execute(query).fetchdf()
print((df.columns))

Index(['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip',
       'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad',
       'mrentabilidad_annual', 'mcomisiones',
       ...
       'delta_1m_mtransferencias_recibidas', 'delta_1m_Visa_mpagominimo',
       'delta_1m_ccomisiones_mantenimiento', 'delta_1m_Visa_mconsumospesos',
       'delta_1m_mcomisiones_otras', 'delta_1m_mcaja_ahorro_dolares',
       'delta_1m_Master_mlimitecompra', 'delta_1m_cmobile_app_trx',
       'delta_1m_Master_mpagominimo', 'delta_1m_ccajas_transacciones'],
      dtype='object', length=339)


# delta_2

In [ ]:
delta_vars = [f"delta_2m_{lag_feature}" for lag_feature in lags]
delta_vars

['delta_2m_ctrx_quarter',
 'delta_2m_mcaja_ahorro',
 'delta_2m_mpasivos_margen',
 'delta_2m_mcuentas_saldo',
 'delta_2m_cpayroll_trx',
 'delta_2m_mprestamos_personales',
 'delta_2m_mtarjeta_visa_consumo',
 'delta_2m_mcuenta_corriente',
 'delta_2m_cprestamos_personales',
 'delta_2m_ctarjeta_visa_transacciones',
 'delta_2m_mrentabilidad_annual',
 'delta_2m_Visa_msaldopesos',
 'delta_2m_mpayroll',
 'delta_2m_Master_status',
 'delta_2m_mactivos_margen',
 'delta_2m_Visa_status',
 'delta_2m_ctarjeta_visa',
 'delta_2m_mcomisiones_mantenimiento',
 'delta_2m_ctarjeta_master',
 'delta_2m_tcallcenter',
 'delta_2m_mcomisiones',
 'delta_2m_Visa_msaldototal',
 'delta_2m_chomebanking_transacciones',
 'delta_2m_cproductos',
 'delta_2m_internet',
 'delta_2m_Master_Fvencimiento',
 'delta_2m_ccallcenter_transacciones',
 'delta_2m_ccajas_consultas',
 'delta_2m_Master_mfinanciacion_limite',
 'delta_2m_mautoservicio',
 'delta_2m_mrentabilidad',
 'delta_2m_Visa_mpagospesos',
 'delta_2m_Visa_Fvencimiento',
 '

In [ ]:
str_delta = ',\n'.join([
    f"{lag_feature} - lag_2_{lag_feature} as delta_2m_{lag_feature}"
    for lag_feature in lags
])
str_delta

'ctrx_quarter - lag_2_ctrx_quarter as delta_2m_ctrx_quarter,\nmcaja_ahorro - lag_2_mcaja_ahorro as delta_2m_mcaja_ahorro,\nmpasivos_margen - lag_2_mpasivos_margen as delta_2m_mpasivos_margen,\nmcuentas_saldo - lag_2_mcuentas_saldo as delta_2m_mcuentas_saldo,\ncpayroll_trx - lag_2_cpayroll_trx as delta_2m_cpayroll_trx,\nmprestamos_personales - lag_2_mprestamos_personales as delta_2m_mprestamos_personales,\nmtarjeta_visa_consumo - lag_2_mtarjeta_visa_consumo as delta_2m_mtarjeta_visa_consumo,\nmcuenta_corriente - lag_2_mcuenta_corriente as delta_2m_mcuenta_corriente,\ncprestamos_personales - lag_2_cprestamos_personales as delta_2m_cprestamos_personales,\nctarjeta_visa_transacciones - lag_2_ctarjeta_visa_transacciones as delta_2m_ctarjeta_visa_transacciones,\nmrentabilidad_annual - lag_2_mrentabilidad_annual as delta_2m_mrentabilidad_annual,\nVisa_msaldopesos - lag_2_Visa_msaldopesos as delta_2m_Visa_msaldopesos,\nmpayroll - lag_2_mpayroll as delta_2m_mpayroll,\nMaster_status - lag_2_Mast

In [ ]:
query = f"""create or replace table competencia_01 as
select *,
    {str_delta}
from competencia_01"""

In [ ]:
con.execute(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
query = """select * from competencia_01 limit 5"""
df = con.execute(query).fetchdf()
print((df.columns))

Index(['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip',
       'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad',
       'mrentabilidad_annual', 'mcomisiones',
       ...
       'delta_2m_mtransferencias_recibidas', 'delta_2m_Visa_mpagominimo',
       'delta_2m_ccomisiones_mantenimiento', 'delta_2m_Visa_mconsumospesos',
       'delta_2m_mcomisiones_otras', 'delta_2m_mcaja_ahorro_dolares',
       'delta_2m_Master_mlimitecompra', 'delta_2m_cmobile_app_trx',
       'delta_2m_Master_mpagominimo', 'delta_2m_ccajas_transacciones'],
      dtype='object', length=385)


# delta_3

In [ ]:
delta_vars = [f"delta_3m_{lag_feature}" for lag_feature in lags]
delta_vars

['delta_3m_ctrx_quarter',
 'delta_3m_mcaja_ahorro',
 'delta_3m_mpasivos_margen',
 'delta_3m_mcuentas_saldo',
 'delta_3m_cpayroll_trx',
 'delta_3m_mprestamos_personales',
 'delta_3m_mtarjeta_visa_consumo',
 'delta_3m_mcuenta_corriente',
 'delta_3m_cprestamos_personales',
 'delta_3m_ctarjeta_visa_transacciones',
 'delta_3m_mrentabilidad_annual',
 'delta_3m_Visa_msaldopesos',
 'delta_3m_mpayroll',
 'delta_3m_Master_status',
 'delta_3m_mactivos_margen',
 'delta_3m_Visa_status',
 'delta_3m_ctarjeta_visa',
 'delta_3m_mcomisiones_mantenimiento',
 'delta_3m_ctarjeta_master',
 'delta_3m_tcallcenter',
 'delta_3m_mcomisiones',
 'delta_3m_Visa_msaldototal',
 'delta_3m_chomebanking_transacciones',
 'delta_3m_cproductos',
 'delta_3m_internet',
 'delta_3m_Master_Fvencimiento',
 'delta_3m_ccallcenter_transacciones',
 'delta_3m_ccajas_consultas',
 'delta_3m_Master_mfinanciacion_limite',
 'delta_3m_mautoservicio',
 'delta_3m_mrentabilidad',
 'delta_3m_Visa_mpagospesos',
 'delta_3m_Visa_Fvencimiento',
 '

In [ ]:
str_delta = ',\n'.join([
    f"{lag_feature} - lag_3_{lag_feature} as delta_3m_{lag_feature}"
    for lag_feature in lags
])
str_delta

'ctrx_quarter - lag_3_ctrx_quarter as delta_3m_ctrx_quarter,\nmcaja_ahorro - lag_3_mcaja_ahorro as delta_3m_mcaja_ahorro,\nmpasivos_margen - lag_3_mpasivos_margen as delta_3m_mpasivos_margen,\nmcuentas_saldo - lag_3_mcuentas_saldo as delta_3m_mcuentas_saldo,\ncpayroll_trx - lag_3_cpayroll_trx as delta_3m_cpayroll_trx,\nmprestamos_personales - lag_3_mprestamos_personales as delta_3m_mprestamos_personales,\nmtarjeta_visa_consumo - lag_3_mtarjeta_visa_consumo as delta_3m_mtarjeta_visa_consumo,\nmcuenta_corriente - lag_3_mcuenta_corriente as delta_3m_mcuenta_corriente,\ncprestamos_personales - lag_3_cprestamos_personales as delta_3m_cprestamos_personales,\nctarjeta_visa_transacciones - lag_3_ctarjeta_visa_transacciones as delta_3m_ctarjeta_visa_transacciones,\nmrentabilidad_annual - lag_3_mrentabilidad_annual as delta_3m_mrentabilidad_annual,\nVisa_msaldopesos - lag_3_Visa_msaldopesos as delta_3m_Visa_msaldopesos,\nmpayroll - lag_3_mpayroll as delta_3m_mpayroll,\nMaster_status - lag_3_Mast

In [ ]:
query = f"""create or replace table competencia_01 as
select *,
    {str_delta}
from competencia_01"""

In [ ]:
con.execute(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
query = """select * from competencia_01 limit 5"""
df = con.execute(query).fetchdf()
print((df.columns))

Index(['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip',
       'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad',
       'mrentabilidad_annual', 'mcomisiones',
       ...
       'delta_3m_mtransferencias_recibidas', 'delta_3m_Visa_mpagominimo',
       'delta_3m_ccomisiones_mantenimiento', 'delta_3m_Visa_mconsumospesos',
       'delta_3m_mcomisiones_otras', 'delta_3m_mcaja_ahorro_dolares',
       'delta_3m_Master_mlimitecompra', 'delta_3m_cmobile_app_trx',
       'delta_3m_Master_mpagominimo', 'delta_3m_ccajas_transacciones'],
      dtype='object', length=431)


# ratios

In [ ]:
ratio_feats = ',\n'.join([
    f"{lag_feature} / nullif(lag_1_{lag_feature}, 0) as ratio_{lag_feature}_vs_prev"
    for lag_feature in lags
])

ratio_vars = [f"ratio_{lag_feature}_vs_prev" for lag_feature in lags]
ratio_vars

['ratio_ctrx_quarter_vs_prev',
 'ratio_mcaja_ahorro_vs_prev',
 'ratio_mpasivos_margen_vs_prev',
 'ratio_mcuentas_saldo_vs_prev',
 'ratio_cpayroll_trx_vs_prev',
 'ratio_mprestamos_personales_vs_prev',
 'ratio_mtarjeta_visa_consumo_vs_prev',
 'ratio_mcuenta_corriente_vs_prev',
 'ratio_cprestamos_personales_vs_prev',
 'ratio_ctarjeta_visa_transacciones_vs_prev',
 'ratio_mrentabilidad_annual_vs_prev',
 'ratio_Visa_msaldopesos_vs_prev',
 'ratio_mpayroll_vs_prev',
 'ratio_Master_status_vs_prev',
 'ratio_mactivos_margen_vs_prev',
 'ratio_Visa_status_vs_prev',
 'ratio_ctarjeta_visa_vs_prev',
 'ratio_mcomisiones_mantenimiento_vs_prev',
 'ratio_ctarjeta_master_vs_prev',
 'ratio_tcallcenter_vs_prev',
 'ratio_mcomisiones_vs_prev',
 'ratio_Visa_msaldototal_vs_prev',
 'ratio_chomebanking_transacciones_vs_prev',
 'ratio_cproductos_vs_prev',
 'ratio_internet_vs_prev',
 'ratio_Master_Fvencimiento_vs_prev',
 'ratio_ccallcenter_transacciones_vs_prev',
 'ratio_ccajas_consultas_vs_prev',
 'ratio_Master_mfi

In [ ]:
query = f"""create or replace table competencia_01 as
select *,
    {ratio_feats}
from competencia_01"""

In [ ]:
con.execute(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
query = """select * from competencia_01 limit 5"""
df = con.execute(query).fetchdf()
print((df.columns))

Index(['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip',
       'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad',
       'mrentabilidad_annual', 'mcomisiones',
       ...
       'ratio_mtransferencias_recibidas_vs_prev',
       'ratio_Visa_mpagominimo_vs_prev',
       'ratio_ccomisiones_mantenimiento_vs_prev',
       'ratio_Visa_mconsumospesos_vs_prev', 'ratio_mcomisiones_otras_vs_prev',
       'ratio_mcaja_ahorro_dolares_vs_prev',
       'ratio_Master_mlimitecompra_vs_prev', 'ratio_cmobile_app_trx_vs_prev',
       'ratio_Master_mpagominimo_vs_prev',
       'ratio_ccajas_transacciones_vs_prev'],
      dtype='object', length=477)


# estandarizadas

In [ ]:
vars_to_standardize = [
    "ctrx_quarter",
    "mcuentas_saldo",
    "mcaja_ahorro",
    "mprestamos_personales",
    "mpasivos_margen",
    "mtarjeta_visa_consumo",
    "mpayroll",
    "mrentabilidad_annual",
    "mcomisiones",
    "mcuenta_corriente",
    "mactivos_margen",
    "mcomisiones_mantenimiento",
    "mcomisiones_otras",
    "cproductos",
    "cdescubierto_preacordado",
    "cpayroll_trx",
    "ccomisiones_mantenimiento",
    "ccallcenter_transacciones",
    "ccajas_consultas",
    "ctarjeta_visa_transacciones",
    "Master_status",
    "Master_mfinanciacion_limite",
    "Visa_status",
    "Visa_msaldototal",
    "mtransferencias_recibidas",
    "mextraccion_autoservicio",
    "mcuenta_debitos_automaticos",
    "mpagomiscuentas",
    "mautoservicio",
    "Visa_msaldopesos",
    "Visa_mlimitecompra",
    "Visa_mpagospesos",
    "Master_msaldopesos",
    "Master_mlimitecompra",
    "Master_mconsumototal",
    "Visa_mconsumototal",
    "Visa_mconsumospesos",
    "Visa_madelantopesos",
    "Master_mfinanciacion_limite",
    "mcuenta_corriente"
]
vars_to_standardize = lags[0:15]

In [ ]:
std_features = [f'std_{var}' for var in vars_to_standardize]

str_std = ',\n'.join([
    f'({var} - avg({var}) over (partition by foto_mes)) / nullif(stddev_pop({var}) over (partition by foto_mes),0) as std_{var}'
    for var in vars_to_standardize
])


In [ ]:
query = f"""create or replace table competencia_01 as
select *,
    {str_std}
from competencia_01"""

In [ ]:
con.execute(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
query = """select * from competencia_01 limit 5"""
df = con.execute(query).fetchdf()
print((list(df.columns)))

['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'csegur

# data drift

In [ ]:
# query = """SELECT
#     foto_mes,
#     AVG(mpayroll) AS mean_payroll,
#     STDDEV(mpayroll) AS std_payroll,
#     AVG(mcuentas_saldo) AS mean_saldo
# FROM competencia_01
# GROUP BY foto_mes
# ORDER BY foto_mes;"""
# df = con.execute(query).fetchdf()
# df

## es_empleado

In [ ]:
query = """CREATE OR REPLACE TABLE competencia_01 AS
SELECT
    *,
    CASE
        WHEN COALESCE(cpayroll_trx,0) > 0 OR COALESCE(mpayroll,0) > 0 THEN 1
        ELSE 0
    END AS es_empleado
FROM competencia_01;"""


In [ ]:
#con.execute(query)

In [ ]:
# query = """select * from competencia_01 limit 5"""
# df = con.execute(query).fetchdf()
# "es_empleado" in df.columns

## percentil

In [ ]:
percent_vars = lags[0:10]

percentil_sql = ',\n'.join([
    f"""CASE
            WHEN {v} > 0 THEN PERCENT_RANK() OVER (PARTITION BY foto_mes ORDER BY {v})
            WHEN {v} < 0 THEN -1 * PERCENT_RANK() OVER (PARTITION BY foto_mes ORDER BY {v} DESC)
            ELSE 0
        END AS {v}_ranked"""
    for v in percent_vars
])

# Armar el bloque SQL completo
query = f"""
CREATE OR REPLACE TABLE competencia_01 AS
SELECT
    *,
    {percentil_sql}
FROM competencia_01;
"""

In [ ]:
#con.execute(query)

In [ ]:
query = """select * from competencia_01 limit 5"""
df = con.execute(query).fetchdf()
df.columns

Index(['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip',
       'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad',
       'mrentabilidad_annual', 'mcomisiones',
       ...
       'std_mprestamos_personales', 'std_mtarjeta_visa_consumo',
       'std_mcuenta_corriente', 'std_cprestamos_personales',
       'std_ctarjeta_visa_transacciones', 'std_mrentabilidad_annual',
       'std_Visa_msaldopesos', 'std_mpayroll', 'std_Master_status',
       'std_mactivos_margen'],
      dtype='object', length=492)

# pendiente

In [ ]:
slope_feats = ',\n'.join([
    f"({lag_feature} - lag_2_{lag_feature}) / 2.0 as slope_2m_{lag_feature}"
    for lag_feature in lags
])

slope_vars = [f"slope_2m_{lag_feature}" for lag_feature in lags]
str_slope = "\n"
print(slope_vars)

['slope_2m_ctrx_quarter', 'slope_2m_mcaja_ahorro', 'slope_2m_mpasivos_margen', 'slope_2m_mcuentas_saldo', 'slope_2m_cpayroll_trx', 'slope_2m_mprestamos_personales', 'slope_2m_mtarjeta_visa_consumo', 'slope_2m_mcuenta_corriente', 'slope_2m_cprestamos_personales', 'slope_2m_ctarjeta_visa_transacciones', 'slope_2m_mrentabilidad_annual', 'slope_2m_Visa_msaldopesos', 'slope_2m_mpayroll', 'slope_2m_Master_status', 'slope_2m_mactivos_margen', 'slope_2m_Visa_status', 'slope_2m_ctarjeta_visa', 'slope_2m_mcomisiones_mantenimiento', 'slope_2m_ctarjeta_master', 'slope_2m_tcallcenter', 'slope_2m_mcomisiones', 'slope_2m_Visa_msaldototal', 'slope_2m_chomebanking_transacciones', 'slope_2m_cproductos', 'slope_2m_internet', 'slope_2m_Master_Fvencimiento', 'slope_2m_ccallcenter_transacciones', 'slope_2m_ccajas_consultas', 'slope_2m_Master_mfinanciacion_limite', 'slope_2m_mautoservicio', 'slope_2m_mrentabilidad', 'slope_2m_Visa_mpagospesos', 'slope_2m_Visa_Fvencimiento', 'slope_2m_ccomisiones_otras', 'slo

In [ ]:
query = f"""create or replace table competencia_01 as
select *,
    {slope_feats}
from competencia_01"""

In [ ]:
# con.execute(query)

In [ ]:
slope_vars

['slope_2m_ctrx_quarter',
 'slope_2m_mcaja_ahorro',
 'slope_2m_mpasivos_margen',
 'slope_2m_mcuentas_saldo',
 'slope_2m_cpayroll_trx',
 'slope_2m_mprestamos_personales',
 'slope_2m_mtarjeta_visa_consumo',
 'slope_2m_mcuenta_corriente',
 'slope_2m_cprestamos_personales',
 'slope_2m_ctarjeta_visa_transacciones',
 'slope_2m_mrentabilidad_annual',
 'slope_2m_Visa_msaldopesos',
 'slope_2m_mpayroll',
 'slope_2m_Master_status',
 'slope_2m_mactivos_margen',
 'slope_2m_Visa_status',
 'slope_2m_ctarjeta_visa',
 'slope_2m_mcomisiones_mantenimiento',
 'slope_2m_ctarjeta_master',
 'slope_2m_tcallcenter',
 'slope_2m_mcomisiones',
 'slope_2m_Visa_msaldototal',
 'slope_2m_chomebanking_transacciones',
 'slope_2m_cproductos',
 'slope_2m_internet',
 'slope_2m_Master_Fvencimiento',
 'slope_2m_ccallcenter_transacciones',
 'slope_2m_ccajas_consultas',
 'slope_2m_Master_mfinanciacion_limite',
 'slope_2m_mautoservicio',
 'slope_2m_mrentabilidad',
 'slope_2m_Visa_mpagospesos',
 'slope_2m_Visa_Fvencimiento',
 '

# mean

In [ ]:
mean_feats = ',\n'.join([
    f"avg({lag_feature}) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and 1 preceding) as mean_{lag_feature}"
    for lag_feature in lags
])

mean_vars = [f"mean_{lag_feature}" for lag_feature in lags]

mean_vars

['mean_ctrx_quarter',
 'mean_mcaja_ahorro',
 'mean_mpasivos_margen',
 'mean_mcuentas_saldo',
 'mean_cpayroll_trx',
 'mean_mprestamos_personales',
 'mean_mtarjeta_visa_consumo',
 'mean_mcuenta_corriente',
 'mean_cprestamos_personales',
 'mean_ctarjeta_visa_transacciones',
 'mean_mrentabilidad_annual',
 'mean_Visa_msaldopesos',
 'mean_mpayroll',
 'mean_Master_status',
 'mean_mactivos_margen',
 'mean_Visa_status',
 'mean_ctarjeta_visa',
 'mean_mcomisiones_mantenimiento',
 'mean_ctarjeta_master',
 'mean_tcallcenter',
 'mean_mcomisiones',
 'mean_Visa_msaldototal',
 'mean_chomebanking_transacciones',
 'mean_cproductos',
 'mean_internet',
 'mean_Master_Fvencimiento',
 'mean_ccallcenter_transacciones',
 'mean_ccajas_consultas',
 'mean_Master_mfinanciacion_limite',
 'mean_mautoservicio',
 'mean_mrentabilidad',
 'mean_Visa_mpagospesos',
 'mean_Visa_Fvencimiento',
 'mean_ccomisiones_otras',
 'mean_Visa_mfinanciacion_limite',
 'mean_cdescubierto_preacordado',
 'mean_mtransferencias_recibidas',
 'm

In [ ]:
query = f"""create or replace table competencia_01 as
select *,
    {mean_feats}
from competencia_01"""



In [ ]:
#con.execute(query)

In [ ]:
query = """select * from competencia_01 limit 5"""
df = con.execute(query).fetchdf()
print((list(df.columns)))

['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'csegur

# max

In [ ]:
max_feats = ',\n'.join([
    f"max({lag_feature}) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and 1 preceding) as max_3m_{lag_feature}"
    for lag_feature in lags
])

max_vars = [f"max_{lag_feature}" for lag_feature in lags]


In [ ]:
query = f"""create or replace table competencia_01 as
select *,
    {max_feats}
from competencia_01"""


In [ ]:
#con.execute(query)

In [ ]:
query = """select * from competencia_01 limit 5"""
df = con.execute(query).fetchdf()
print((list(df.columns)))

['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'csegur

# min

In [ ]:
min_feats = ',\n'.join([
    f"min({lag_feature}) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and 1 preceding) as min_{lag_feature}"
    for lag_feature in lags
])

min_vars = [f"min_{lag_feature}" for lag_feature in lags]


In [ ]:
query = f"""create or replace table competencia_01 as
select *,
    {min_feats}
from competencia_01"""

In [ ]:
#con.execute(query)

In [ ]:
query = """select * from competencia_01 limit 5"""
df = con.execute(query).fetchdf()
print((list(df.columns)))

['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'csegur

# tc_saldo_total

In [ ]:
# Crea campo tc_saldo_total con la suma del saldo en tarjeta Master y tarjeta Visa

query = """create or replace table competencia_01 as
select *,
    ifnull(Master_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tc_saldo_total
from competencia_01"""

# con.execute(query)

In [ ]:
query = """select * from competencia_01 limit 5"""
df = con.execute(query).fetchdf()
print((list(df.columns)))

['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'csegur

# guardo archivo

In [ ]:
cols_no_lags = [col for col in df.columns if "lag" not in col]

In [ ]:
query = f'select {"\n,".join(cols_no_lags)} from competencia_01'

In [ ]:
query = f'select * from competencia_01'

In [ ]:
con.execute(f"copy ({query}) to '/content/drive/MyDrive/EyF/fe_muerte_v6.parquet' (format parquet)")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
query = f"""select * from competencia_01 limit 5"""
df = con.execute(query).fetchdf()
print(list(df.columns))

['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'csegur

In [ ]:
len(df.columns)

354